In [1]:
import random
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import numpy as np
import torch.utils.data
import cv2
import torchvision.models.segmentation
import torch
import os
import patchify
from sklearn.datasets import load_sample_image
from sklearn.feature_extraction import image as skimg
import imgaug as ia
import imgaug.augmenters as iaa
from torchvision.models import resnet50, ResNet50_Weights
from torchvision.models.detection.mask_rcnn import MaskRCNN_ResNet50_FPN_V2_Weights
import torch.optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.insert(1, '/home/prakharug/Experimental')
sys.path.insert(1, '/home/prakharug/Experimental/pycoco')
from pycoco.engine import train_one_epoch, evaluate

In [2]:
def combine_dims(a, start=0, count=2):
    """ Reshapes numpy array a by combining count dimensions, 
        starting at dimension index start """
    s = a.shape
    return np.reshape(a, s[:start] + (-1,) + s[start+count:])


In [3]:
instance_toimage = []

for im_name in os.listdir("../dataset/images"):
    tmplist = []
    lent = len(im_name[:-4])
    print(im_name,im_name[:lent])
    imgtmp = cv2.imread("../dataset/images/"+im_name,1).transpose(2,0,1)
    #print(imgtmp[0].shape)
    tmplist.append(imgtmp[0])
    tmplist.append(imgtmp[1])
    tmplist.append(imgtmp[2])
    for gt_name in os.listdir("../dataset/ground_truths"):
        #print(gt_name[10+lent],gt_name[10:10+lent])
        if gt_name[10:10+lent]==im_name[:lent] and gt_name[10+lent]=="_":
            mask = np.array(cv2.imread("../dataset/ground_truths/"+gt_name,0))
            # mask = (mask > 0).astype(np.uint8) 
            tmplist.append(mask)
    tmplist = np.array(tmplist)
    print(tmplist.shape)
    instance_toimage.append(tmplist)

# for im_name in os.listdir("../dataset/test"):
#     tmplist = []
#     lent = len(im_name[:-4])
#     print(im_name,im_name[:lent])
#     imgtmp = cv2.imread("../dataset/test/"+im_name,1).transpose(2,0,1)
#     #print(imgtmp[0].shape)
#     tmplist.append(imgtmp[0])
#     tmplist.append(imgtmp[1])
#     tmplist.append(imgtmp[2])
#     for gt_name in os.listdir("../dataset/ground_truths"):
#         #print(gt_name[10+lent],gt_name[10:10+lent])
#         if gt_name[10:10+lent]==im_name[:lent] and gt_name[10+lent]=="_":
#             mask = np.array(cv2.imread("../dataset/ground_truths/"+gt_name,0))
#             # mask = (mask > 0).astype(np.uint8) 
#             tmplist.append(mask)
#     tmplist = np.array(tmplist)
#     print(tmplist.shape)
#     instance_toimage.append(tmplist)



FileNotFoundError: [Errno 2] No such file or directory: '../dataset/images'

In [ ]:
batchSize = 2
img_siz = [512,512]
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 
print(device)

from readline import append_history_file
from unittest.mock import patch


def loadBatch():
    rtrnd=random.randint(0,4)
    seq = iaa.Sequential([
    iaa.Sometimes(
        0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    iaa.Rot90((rtrnd), keep_size=False)
    ])
    batch_Imgs=[]
    batch_Data=[]
    for i in range(batchSize):
        idx=random.randint(0,len(instance_toimage)-1)
        patch_stack = seq(images=instance_toimage[idx])
        patch_stack = np.array(patch_stack)
        k_w = random.randint(512,patch_stack.shape[2])
        k_h = random.randint(512,patch_stack.shape[1])
        #k_w = patch_stack.shape[2]
        #k_h = patch_stack.shape[1]
        o_w = random.randint(0,patch_stack.shape[2]-k_w)
        o_h = random.randint(0,patch_stack.shape[1]-k_h)
        patch_img = patch_stack[0:3,o_h:o_h+k_h,o_w:o_w+k_w]
        instances = patch_stack[3:,o_h:o_h+k_h,o_w:o_w+k_w]
        instances = instances.transpose(1,2,0)
        data = {}
        masks = []
        boxes = []
        t=0
        sem_mask = np.zeros((instances.shape[0],instances.shape[1]))
        for a in range(instances.shape[2]):
            dispim = instances[:,:,a]
            if np.all(dispim == 0):
                continue
            x,y,w,h = cv2.boundingRect(dispim)
            boxes.append([x, y, x+w, y+h])
            masks.append(dispim/255)
            sem_mask += dispim
            t=1
        masks = np.array(masks)
        sem_mask = sem_mask > 0
        sem_mask_c = torch.as_tensor(sem_mask.sum())
        neg_mask = (sem_mask == 0)
        neg_mask_c = torch.as_tensor(neg_mask.sum())
        sem_mask = np.array([sem_mask,sem_mask,sem_mask])
        sem_mask = torch.as_tensor(sem_mask,dtype=torch.uint8)
        neg_mask = np.array([neg_mask,neg_mask,neg_mask])
        neg_mask = torch.as_tensor(neg_mask,dtype=torch.uint8)
        if t==0:
            return loadBatch()
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        img = torch.as_tensor(patch_img, dtype=torch.float32)
        img = img/255
        data["boxes"] =  boxes
        data["labels"] =  torch.ones((boxes.shape[0],), dtype=torch.int64)   # there is only one class
        data["masks"] = masks
        data["sem_mask"] = sem_mask
        data["neg_mask"] = neg_mask
        data["sem_mask_c"] = sem_mask_c
        data["neg_mask_c"] = neg_mask_c
        batch_Imgs.append(img)
        batch_Data.append(data)
    return batch_Imgs, batch_Data



In [ ]:
class MMCellDataset(Dataset):
    def __init__(self,root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.img = []
        for im_name in os.listdir(self.root_dir):
            tmplist = []
            lent = len(im_name[:-4])
            print(im_name,im_name[:lent])
            imgtmp = cv2.imread(self.root_dir+im_name,1).transpose(2,0,1)
            tmplist.append(imgtmp[0])
            tmplist.append(imgtmp[1])
            tmplist.append(imgtmp[2])
            for gt_name in os.listdir("../dataset/ground_truths"):
                if gt_name[10:10+lent]==im_name[:lent] and gt_name[10+lent]=="_":
                    mask = np.array(cv2.imread("../dataset/ground_truths/"+gt_name,0))
                    tmplist.append(mask)
            tmplist = np.array(tmplist)
            print(tmplist.shape)
            self.img.append(tmplist)   
    
    def __len__(self):
        return len([name for name in os.listdir(self.root_dir)])

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        patch_stack = self.img[idx]
        patch_stack = np.array(patch_stack)
        k_w = patch_stack.shape[2]
        k_h = patch_stack.shape[1]
        o_w = random.randint(0,patch_stack.shape[2]-k_w)
        o_h = random.randint(0,patch_stack.shape[1]-k_h)
        patch_img = patch_stack[0:3,o_h:o_h+k_h,o_w:o_w+k_w]
        instances = patch_stack[3:,o_h:o_h+k_h,o_w:o_w+k_w]
        instances = instances.transpose(1,2,0)
        data = {}
        masks = []
        boxes = []
        area = []
        t=0
        for a in range(instances.shape[2]):
            dispim = instances[:,:,a]
            x,y,w,h = cv2.boundingRect(dispim)
            boxes.append([x, y, x+w, y+h])
            area.append(torch.tensor(h*w))
            masks.append(dispim/255)
            t=1
        masks = np.array(masks)
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        area = torch.as_tensor(area, dtype=torch.float32)
        img = torch.as_tensor(patch_img, dtype=torch.float32)
        img = img/255
        data["boxes"] =  boxes
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)
        data["iscrowd"] = iscrowd
        data["labels"] =  torch.ones((boxes.shape[0],), dtype=torch.int64)   # there is only one class
        data["masks"] = masks
        data["area"] = area
        data["image_id"] = torch.tensor(idx)
        return img,data
test = MMCellDataset("../dataset/test/")
def collate_fn(batch):
    return tuple(zip(*batch))
print(len(test))
test_dataloader = DataLoader(test, batch_size=2, shuffle=False,collate_fn = collate_fn)


In [ ]:
class jsrcnn_v5(nn.Module):
    def __init__(self,**kwargs) -> None:
        super(jsrcnn_v5, self).__init__()
        self.vis = False
        #self.param = param
        self.channel_trans = nn.ModuleList([
            torch.nn.Conv2d(3,12,(1,1),stride=1,padding='same'),
            #torch.nn.Conv2d(12,24,(1,1),stride=1,padding='same'),
            torch.nn.Conv2d(12,3,(1,1),stride=1,padding='same'),
            #torch.nn.Conv2d(24,12,(72,72),stride=1,padding='same',padding_mode='replicate'),
            #torch.nn.Conv2d(12,12,(48,48),stride=1,padding='same',padding_mode='replicate'),       
        ])
        # self.maskbone = nn.Sequential(
        #     torch.nn.Conv2d(27,12,(3,3),stride=1,padding='same',padding_mode='replicate'),
        #     torch.nn.Conv2d(12,12,(24,24),stride=1,padding='same',padding_mode='replicate'),
        #     torch.nn.Conv2d(12,12,(32,32),stride=1,padding='same',padding_mode='replicate'),
        # )
        self.sigma = nn.Sigmoid()
        self.norm3 = nn.BatchNorm2d(3,eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.norm12 = nn.BatchNorm2d(12,eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.ReLU = nn.ReLU()
        # self.tobi = torch.nn.Conv2d(42,1,(5,5),stride=1,padding='same',padding_mode='replicate')
        # self.rko = torch.nn.Conv2d(15,3,(1,1),stride=1,padding='same',padding_mode='replicate')
        self.maskrcnn = torchvision.models.detection.maskrcnn_resnet50_fpn_v2(weights=MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT,trainable_backbone_layers=5,**kwargs)
        in_features = self.maskrcnn.roi_heads.box_predictor.cls_score.in_features  # get number of input features for the classifier
        self.maskrcnn.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_classes=2)  # replace the pre-trained head with a new one

    def forward(self, images, targets=None):
        trniml = []
        color_loss = torch.Tensor([0]).to(device)
        for ind in range(len(images)):
            img = torch.stack([images[ind]])
            imgog = img
            img = self.channel_trans[0](img)
            img = self.norm12(img)
            img = self.ReLU(img)
            img = self.channel_trans[1](img)
            #img = self.ReLU(img)
            #img = self.channel_trans[2](img)
            img = self.ReLU(self.norm3(img))
            images[ind] = img[0]
            if self.vis == True:
                cv2.imwrite("mod_img.png",(img[0].clone().detach().cpu().numpy()*255).transpose(1,2,0))
                cv2.imwrite("og_img.png",(imgog[0].clone().detach().cpu().numpy()*255).transpose(1,2,0))
            #img3 = self.channel_trans[2](img)
            #img3 = self.ReLU(img3)
            # img3 = img
            # img4 = self.channel_trans[2](img3)
            # img4 = self.ReLU(img4)
            # img5 = self.channel_trans[3](img4)
            # img5 = self.ReLU(img5)
            # imgmask = torch.concat((img5,img4,imgog),1)
            # mk = self.maskbone(imgmask)
            # segmask = self.tobi(torch.concat((imgmask,mk,imgog),1))
            # images[ind] = torch.cat((images[ind],xconv2[0],xconv1[0]),0)
            # H = images[ind].shape[1]
            # W = images[ind].shape[2]
            # kek = images[ind].to(device)
            # images[ind] = images[ind].permute((1,2,0))
            # images[ind] = images[ind].reshape([H*W,12])
            # images[ind] = self.channel_trans(images[ind])
            # images[ind] = images[ind].reshape([H,W,3])
            # images[ind] = images[ind].permute((2,0,1))
        ret = self.maskrcnn(images,targets)
        return ret

In [ ]:
model = jsrcnn_v5(box_nms_thresh=0.5)
model.to(device)
model.load_state_dict(torch.load('jsr_v5.69_1200.torch'))
optimizer = torch.optim.AdamW([{ 'params': model.maskrcnn.parameters(), 'lr' : 1e-6},{ 'params' : model.channel_trans.parameters(), 'lr' : 1e-11}], lr=1e-4)
#optimizer = torch.optim.AdamW(model.parameters(),lr=1e-4)
model.train()
lmbda = lambda epoch: 0.8
# for param in model.channel_trans.parameters():
#     param.requires_grad = False
scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lmbda)

minloss = 10

In [ ]:
test = 1800
for i in range(10000):
    if i%20==0:
        model.vis = True
    images, targets = loadBatch()
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

    optimizer.zero_grad()
    loss_dict = model(images, targets)
    # for losst in loss_dict.keys():
    #     print(losst,loss_dict[losst])
    losses = sum(loss for loss in loss_dict.values())

    losses.backward()
    optimizer.step()
    print(test+i,'loss:', losses.item())#," color_loss:",loss_dict["color_loss"].item())
    model.vis = False
    
    if i%100==0:
        scheduler.step()
        print(scheduler.get_lr())
        evaluate(model, test_dataloader, device=device)
        model.train()
    if i%200==0 and i!=0:
        evaluate(model, test_dataloader, device=device)
        model.train()
        torch.save(model.state_dict(), "jsr_v5.69_"+str(+test+i)+".torch")